In [1]:
from __future__ import annotations
import numpy as np
import random
import copy
import importlib

from typing import Tuple, List
from numpy import array, zeros

# from Big_Class import Big_Class  # already imported one NETfuncs is imported
from User_Variables import User_Variables  # already imported one NETfuncs is imported
from Network_Structure import Network_Structure  # already imported one NETfuncs is imported
from Big_Class import Big_Class
from Network_State import Network_State
from Networkx_Net import Networkx_Net
import matrix_functions, functions, statistics, plot_functions, solve, figure_plots

# Parameters

In [2]:
## Parameters

## task type
# task_type='Iris_classification'
task_type='Regression'

## task matrix X
M_values: np.ndarray = array([0.15, 0.28, 0.15, 0.21, 0.01, 0.3, 0.05, 0.12, 0.23, 0.25, 0.12, 0.1, 0.02, 0.3, 0.35, 0.25,
                              0.03, 0.02, 0.25, 0.31, 0.02, 0.08, 0.35, 0.1, 0.03, 0.2, 0.1, 0.18, 0.12, 0.35, 0.05, 0.15,
                              0.25, 0.25, 0.02, 0.01, 0.08, 0.07, 0.35, 0.15, 0.01, 0.25, 0.13, 0.05, 0.03, 0.3, 0.32, 0.05,
                              0.15, 0.12, 0.25, 0.1, 0.25, 0.3, 0.25, 0.25, 0.04, 0.27, 0.1, 0.11, 0.02, 0.2, 0.4, 0.15,
                             ])

## specify # of nodes
Nin: int = 1
extraNin: int = 0
Ninter: int = 0
Nout: int = 1
extraNout: int = 0

# learning rate
alpha: float = 0.5  # for network combine attempt

# resistance-pressure proportionality factor
gamma: np.ndarray = np.array([1.0])

## method to update resistances - physical property of the system
R_update: str = 'R_propto_dp'
R_vec_i = 2*np.ones(Nin*Nout + Nin + Nout)

# length of training dataset
iterations = 2000  # number of sampled of p

# measure accuracy every # steps
measure_accuracy_every = 15

supress_prints: bool = True  # whether to print information during training or not
bc_noise: float = 0.0  # noise to dual problem
use_p_tag: bool = True  # use 1 or 2 sampled pressures at every time step
include_Power: bool = False
lam = -80**(1)
lam2 = -5*10**(-3)

stay_sample: int = 2  # how many loop iterations to stay under the same sampled p
    
access_interNodes: bool = False  # access and change pressure at interNodes (nodes between input and output) or not
noise_to_extra: bool = False  # add noise to extra outputs 

In [3]:
def network_build_given_Nin_Nout(Nin: int, Nout: int) -> tuple():
    
    # initialize Variables
    Variabs = User_Variables(iterations,\
                         Nin, \
                         extraNin, \
                         Ninter, \
                         Nout, \
                         extraNout, \
                         gamma, \
                         R_update, \
                         use_p_tag, \
                         include_Power, lam, \
                         supress_prints, \
                         bc_noise, \
                         access_interNodes, \
                         task_type, \
                         M_values, \
                         measure_accuracy_every)
    
    Variabs.assign_alpha_vec(alpha)
    Variabs.create_dataset_and_targets(M_values)
    Variabs.create_noise_for_extras()
    BigClass = Big_Class(Variabs)
        
    # initialize Structure
    inInterOutGround_tuple = matrix_functions.build_input_output_and_ground(Variabs.Nin, Variabs.extraNin, Variabs.Ninter, 
                                                                            Variabs.Nout, Variabs.extraNout)
        
    
    Strctr = Network_Structure(inInterOutGround_tuple)
    Strctr.build_incidence()
    Strctr.build_edges()
    BigClass.add_Strctr(Strctr)  # add to big class
    
    # initialize State    
    State = Network_State(Variabs)
    State.initiate_resistances(BigClass, R_vec_i)
    BigClass.add_State(State)  # add to big class
    
    return Variabs, Strctr, State, BigClass

def train_loop(Variabs, Strctr, State, BigClass):
    for l in range(Variabs.iterations):
        k = (l//stay_sample)*2 + l%2

        # draw input and desired outputs from dataset
        if not((l+1) % 4):  # add noise only at i=3 etc.
            State.draw_p_in_and_desired(Variabs, k, noise_to_extra=False)  # add noise to extra nodes every 2nd iteration
            State.solve_flow_given_problem(BigClass, "measure", noise_to_extra=False)  # measure and don't change resistances
        else:  # dont add noise to extra nodes
            State.draw_p_in_and_desired(Variabs, k)
            State.solve_flow_given_problem(BigClass, "measure")

        if not l % 2:  # even iterations, take another sampled pressure and measure again
            pass
        else:  # odd iterations, go to dual problem and update resistances
            State.t += 1
            # print('time=', State.t)
            State.calc_loss(BigClass)
            loss_mean = np.mean(np.abs(State.loss), axis=1)
            State.update_input_dual(BigClass)
            State.update_output_dual(BigClass)
            State.solve_flow_given_problem(BigClass, "dual", access_inters=False)  # measure and don't change resistances
            State.update_Rs(BigClass)
    return State

# Loop

In [4]:
window_for_mean = 100
n = 7
norm_mean_loss = np.zeros([n,n])

for i in range(n):
    for j in range(n):
        Nin: int = i + 1
        Nout: int = j + 1
        
        Variabs, Strctr, State, BigClass = network_build_given_Nin_Nout(Nin, Nout)
        print('starting train loop')
        State = train_loop(Variabs, Strctr, State, BigClass)
        
        mean_loss_ij = np.mean(np.mean(np.abs(State.loss_in_t[-window_for_mean:]), axis=1))
        norm_mean_loss_ij = mean_loss_ij / np.mean(Variabs.targets)
        print('norm_mean_loss_ij', norm_mean_loss_ij)
        
        norm_mean_loss[i,j] = norm_mean_loss_ij

input M mismatches output and input
no extra input nodes, no noise added
no inter nodes, no noise added
no extra output nodes, no noise added
starting train loop
time= 1
time= 2
time= 3
time= 4
time= 5
time= 6
time= 7
time= 8
time= 9
time= 10
time= 11
time= 12
time= 13
time= 14
time= 15
time= 16
time= 17
time= 18
time= 19
time= 20
time= 21
time= 22
time= 23
time= 24
time= 25
time= 26
time= 27
time= 28
time= 29
time= 30
time= 31
time= 32
time= 33
time= 34
time= 35
time= 36
time= 37
time= 38
time= 39
time= 40
time= 41
time= 42
time= 43
time= 44
time= 45
time= 46
time= 47
time= 48
time= 49
time= 50
time= 51
time= 52
time= 53
time= 54
time= 55
time= 56
time= 57
time= 58
time= 59
time= 60
time= 61
time= 62
time= 63
time= 64
time= 65
time= 66
time= 67
time= 68
time= 69
time= 70
time= 71
time= 72
time= 73
time= 74
time= 75
time= 76
time= 77
time= 78
time= 79
time= 80
time= 81
time= 82
time= 83
time= 84
time= 85
time= 86
time= 87
time= 88
time= 89
time= 90
time= 91
time= 92
time= 93
time= 94
t

time= 52
time= 53
time= 54
time= 55
time= 56
time= 57
time= 58
time= 59
time= 60
time= 61
time= 62
time= 63
time= 64
time= 65
time= 66
time= 67
time= 68
time= 69
time= 70
time= 71
time= 72
time= 73
time= 74
time= 75
time= 76
time= 77
time= 78
time= 79
time= 80
time= 81
time= 82
time= 83
time= 84
time= 85
time= 86
time= 87
time= 88
time= 89
time= 90
time= 91
time= 92
time= 93
time= 94
time= 95
time= 96
time= 97
time= 98
time= 99
time= 100
time= 101
time= 102
time= 103
time= 104
time= 105
time= 106
time= 107
time= 108
time= 109
time= 110
time= 111
time= 112
time= 113
time= 114
time= 115
time= 116
time= 117
time= 118
time= 119
time= 120
time= 121
time= 122
time= 123
time= 124
time= 125
time= 126
time= 127
time= 128
time= 129
time= 130
time= 131
time= 132
time= 133
time= 134
time= 135
time= 136
time= 137
time= 138
time= 139
time= 140
time= 141
time= 142
time= 143
time= 144
time= 145
time= 146
time= 147
time= 148
time= 149
time= 150
time= 151
time= 152
time= 153
time= 154
time= 155
time= 15

time= 883
time= 884
time= 885
time= 886
time= 887
time= 888
time= 889
time= 890
time= 891
time= 892
time= 893
time= 894
time= 895
time= 896
time= 897
time= 898
time= 899
time= 900
time= 901
time= 902
time= 903
time= 904
time= 905
time= 906
time= 907
time= 908
time= 909
time= 910
time= 911
time= 912
time= 913
time= 914
time= 915
time= 916
time= 917
time= 918
time= 919
time= 920
time= 921
time= 922
time= 923
time= 924
time= 925
time= 926
time= 927
time= 928
time= 929
time= 930
time= 931
time= 932
time= 933
time= 934
time= 935
time= 936
time= 937
time= 938
time= 939
time= 940
time= 941
time= 942
time= 943
time= 944
time= 945
time= 946
time= 947
time= 948
time= 949
time= 950
time= 951
time= 952
time= 953
time= 954
time= 955
time= 956
time= 957
time= 958
time= 959
time= 960
time= 961
time= 962
time= 963
time= 964
time= 965
time= 966
time= 967
time= 968
time= 969
time= 970
time= 971
time= 972
time= 973
time= 974
time= 975
time= 976
time= 977
time= 978
time= 979
time= 980
time= 981
time= 982


time= 877
time= 878
time= 879
time= 880
time= 881
time= 882
time= 883
time= 884
time= 885
time= 886
time= 887
time= 888
time= 889
time= 890
time= 891
time= 892
time= 893
time= 894
time= 895
time= 896
time= 897
time= 898
time= 899
time= 900
time= 901
time= 902
time= 903
time= 904
time= 905
time= 906
time= 907
time= 908
time= 909
time= 910
time= 911
time= 912
time= 913
time= 914
time= 915
time= 916
time= 917
time= 918
time= 919
time= 920
time= 921
time= 922
time= 923
time= 924
time= 925
time= 926
time= 927
time= 928
time= 929
time= 930
time= 931
time= 932
time= 933
time= 934
time= 935
time= 936
time= 937
time= 938
time= 939
time= 940
time= 941
time= 942
time= 943
time= 944
time= 945
time= 946
time= 947
time= 948
time= 949
time= 950
time= 951
time= 952
time= 953
time= 954
time= 955
time= 956
time= 957
time= 958
time= 959
time= 960
time= 961
time= 962
time= 963
time= 964
time= 965
time= 966
time= 967
time= 968
time= 969
time= 970
time= 971
time= 972
time= 973
time= 974
time= 975
time= 976


time= 690
time= 691
time= 692
time= 693
time= 694
time= 695
time= 696
time= 697
time= 698
time= 699
time= 700
time= 701
time= 702
time= 703
time= 704
time= 705
time= 706
time= 707
time= 708
time= 709
time= 710
time= 711
time= 712
time= 713
time= 714
time= 715
time= 716
time= 717
time= 718
time= 719
time= 720
time= 721
time= 722
time= 723
time= 724
time= 725
time= 726
time= 727
time= 728
time= 729
time= 730
time= 731
time= 732
time= 733
time= 734
time= 735
time= 736
time= 737
time= 738
time= 739
time= 740
time= 741
time= 742
time= 743
time= 744
time= 745
time= 746
time= 747
time= 748
time= 749
time= 750
time= 751
time= 752
time= 753
time= 754
time= 755
time= 756
time= 757
time= 758
time= 759
time= 760
time= 761
time= 762
time= 763
time= 764
time= 765
time= 766
time= 767
time= 768
time= 769
time= 770
time= 771
time= 772
time= 773
time= 774
time= 775
time= 776
time= 777
time= 778
time= 779
time= 780
time= 781
time= 782
time= 783
time= 784
time= 785
time= 786
time= 787
time= 788
time= 789


...

time= 620
time= 621
time= 622
time= 623
time= 624
time= 625
time= 626
time= 627
time= 628
time= 629
time= 630
time= 631
time= 632
time= 633
time= 634
time= 635
time= 636
time= 637
time= 638
time= 639
time= 640
time= 641
time= 642
time= 643
time= 644
time= 645
time= 646
time= 647
time= 648
time= 649
time= 650
time= 651
time= 652
time= 653
time= 654
time= 655
time= 656
time= 657
time= 658
time= 659
time= 660
time= 661
time= 662
time= 663
time= 664
time= 665
time= 666
time= 667
time= 668
time= 669
time= 670
time= 671
time= 672
time= 673
time= 674
time= 675
time= 676
time= 677
time= 678
time= 679
time= 680
time= 681
time= 682
time= 683
time= 684
time= 685
time= 686
time= 687
time= 688
time= 689
time= 690
time= 691
time= 692
time= 693
time= 694
time= 695
time= 696
time= 697
time= 698
time= 699
time= 700
time= 701
time= 702
time= 703
time= 704
time= 705
time= 706
time= 707
time= 708
time= 709
time= 710
time= 711
time= 712
time= 713
time= 714
time= 715
time= 716
time= 717
time= 718
time= 719


time= 451
time= 452
time= 453
time= 454
time= 455
time= 456
time= 457
time= 458
time= 459
time= 460
time= 461
time= 462
time= 463
time= 464
time= 465
time= 466
time= 467
time= 468
time= 469
time= 470
time= 471
time= 472
time= 473
time= 474
time= 475
time= 476
time= 477
time= 478
time= 479
time= 480
time= 481
time= 482
time= 483
time= 484
time= 485
time= 486
time= 487
time= 488
time= 489
time= 490
time= 491
time= 492
time= 493
time= 494
time= 495
time= 496
time= 497
time= 498
time= 499
time= 500
time= 501
time= 502
time= 503
time= 504
time= 505
time= 506
time= 507
time= 508
time= 509
time= 510
time= 511
time= 512
time= 513
time= 514
time= 515
time= 516
time= 517
time= 518
time= 519
time= 520
time= 521
time= 522
time= 523
time= 524
time= 525
time= 526
time= 527
time= 528
time= 529
time= 530
time= 531
time= 532
time= 533
time= 534
time= 535
time= 536
time= 537
time= 538
time= 539
time= 540
time= 541
time= 542
time= 543
time= 544
time= 545
time= 546
time= 547
time= 548
time= 549
time= 550


time= 299
time= 300
time= 301
time= 302
time= 303
time= 304
time= 305
time= 306
time= 307
time= 308
time= 309
time= 310
time= 311
time= 312
time= 313
time= 314
time= 315
time= 316
time= 317
time= 318
time= 319
time= 320
time= 321
time= 322
time= 323
time= 324
time= 325
time= 326
time= 327
time= 328
time= 329
time= 330
time= 331
time= 332
time= 333
time= 334
time= 335
time= 336
time= 337
time= 338
time= 339
time= 340
time= 341
time= 342
time= 343
time= 344
time= 345
time= 346
time= 347
time= 348
time= 349
time= 350
time= 351
time= 352
time= 353
time= 354
time= 355
time= 356
time= 357
time= 358
time= 359
time= 360
time= 361
time= 362
time= 363
time= 364
time= 365
time= 366
time= 367
time= 368
time= 369
time= 370
time= 371
time= 372
time= 373
time= 374
time= 375
time= 376
time= 377
time= 378
time= 379
time= 380
time= 381
time= 382
time= 383
time= 384
time= 385
time= 386
time= 387
time= 388
time= 389
time= 390
time= 391
time= 392
time= 393
time= 394
time= 395
time= 396
time= 397
time= 398


time= 123
time= 124
time= 125
time= 126
time= 127
time= 128
time= 129
time= 130
time= 131
time= 132
time= 133
time= 134
time= 135
time= 136
time= 137
time= 138
time= 139
time= 140
time= 141
time= 142
time= 143
time= 144
time= 145
time= 146
time= 147
time= 148
time= 149
time= 150
time= 151
time= 152
time= 153
time= 154
time= 155
time= 156
time= 157
time= 158
time= 159
time= 160
time= 161
time= 162
time= 163
time= 164
time= 165
time= 166
time= 167
time= 168
time= 169
time= 170
time= 171
time= 172
time= 173
time= 174
time= 175
time= 176
time= 177
time= 178
time= 179
time= 180
time= 181
time= 182
time= 183
time= 184
time= 185
time= 186
time= 187
time= 188
time= 189
time= 190
time= 191
time= 192
time= 193
time= 194
time= 195
time= 196
time= 197
time= 198
time= 199
time= 200
time= 201
time= 202
time= 203
time= 204
time= 205
time= 206
time= 207
time= 208
time= 209
time= 210
time= 211
time= 212
time= 213
time= 214
time= 215
time= 216
time= 217
time= 218
time= 219
time= 220
time= 221
time= 222


time= 100
time= 101
time= 102
time= 103
time= 104
time= 105
time= 106
time= 107
time= 108
time= 109
time= 110
time= 111
time= 112
time= 113
time= 114
time= 115
time= 116
time= 117
time= 118
time= 119
time= 120
time= 121
time= 122
time= 123
time= 124
time= 125
time= 126
time= 127
time= 128
time= 129
time= 130
time= 131
time= 132
time= 133
time= 134
time= 135
time= 136
time= 137
time= 138
time= 139
time= 140
time= 141
time= 142
time= 143
time= 144
time= 145
time= 146
time= 147
time= 148
time= 149
time= 150
time= 151
time= 152
time= 153
time= 154
time= 155
time= 156
time= 157
time= 158
time= 159
time= 160
time= 161
time= 162
time= 163
time= 164
time= 165
time= 166
time= 167
time= 168
time= 169
time= 170
time= 171
time= 172
time= 173
time= 174
time= 175
time= 176
time= 177
time= 178
time= 179
time= 180
time= 181
time= 182
time= 183
time= 184
time= 185
time= 186
time= 187
time= 188
time= 189
time= 190
time= 191
time= 192
time= 193
time= 194
time= 195
time= 196
time= 197
time= 198
time= 199


In [8]:
save_folder_prelim = 'C:/Users/SMR_Admin/OneDrive - huji.ac.il/PhD/Network Simulation repo/Network combine/Network_combine/'

np.save(save_folder_prelim + 'loss_mat.npy', norm_mean_loss)